<h4>Obtaining Latitude and Longitude Values of Directory Locations</h4>

<h5>Import Dependencies</h5>

In [1]:
!pip install w3lib
!pip install selenium
!pip install geocoder
!pip install geopy
!pip install folium

import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
import requests
import csv
import pprint as pp
import time
import re
import urllib.parse

from w3lib.url import url_query_parameter
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

# from selenium.common.exceptions import IndexError

from bs4 import BeautifulSoup as bs  # Beautiful Soup v4

# Only use when full dataframe needs to be viewed
# pd.set_option("display.max_rows", None, "display.max_columns", None)

import geocoder  # import geocoder

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium  # plotting library

print('Libraries imported.')

Libraries imported.


<h5>Extract Contents of Excel File into DataFrame</h5>

In [2]:
akl_cbd_biz_extract = pd.read_excel("url_covid_classifier.xlsx")

akl_cbd_biz = akl_cbd_biz_extract.replace(np.nan, '', regex=True)

k = akl_cbd_biz.shape[0]

akl_cbd_biz.head()

,Name,Main Category,Sub Category 1,Sub Category 2,Address Line 1,Address Line 2,Precinct,Post Code,Locality,Phone,Web URL,Web URL COVID Info
0,100 Fix,Retail,Electronics and technology,,,19 Shortland Street,\n\nFederal & Elliott \n\n,1010,Auckland,,,No URL
1,109 Doctors,Services,Health and wellbeing,Doctors,Level 9,109 Queen Street,\n\nQueen Street \n\n,,Auckland,(09)3660109,,No URL
2,16 Tun,Hospitality,Bars and clubs,Restaurants,,10/26 Jellicoe Street,\n\nViaduct & Waterfront \n\n,,Auckland,09 368 7712,www.16tun.com,URL Parse Error
3,1Above Ltd,Services,Health and wellbeing,,Old Sofrana House Level 1,18-Customs Street East,\n\nViaduct & Waterfront \n\n,,Auckland,09 379 5708,www.fly1above.com,No Obvious COVID Info
4,2 Degrees,Retail,Electronics and technology,,Civic Centre,8 Wellesley Street West,\n\nFederal & Elliott \n\n,,Auckland,09 366 1584,www.2degreesmobile.co.nz,No Obvious COVID Info


<h5>Set Up and Run Latitude and Longitude Information Loop</h5>

In [3]:
address_l = []
latitude_l = []
longitude_l = []

c = 0
for c in range(0, k):
    # extract street address from DataFrame
    addy = akl_cbd_biz['Address Line 2'][c]
    
    # Perform data cleaning
    substr = addy.split()     # Get individual words 
    string = " ".join(sorted(set(substr), key=substr.index))    # Remove repeated words and multi-spaces
    if string[0].isdigit():            
        addy = string
    else:
        addy = re.split(r'(^[^\d]+)', string)[1:][1]   # Remove leading text found in string
        
    # Perform other desired data cleaning steps e.g. remove dashes
    
    address = addy + ', Auckland 1010'                 # Add City and Postcode
      
    # Obtain Latitude and Longitude Values
    try:
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude

    except:
        latitude = 'error'
        longitude = 'error'

    address_l.append(address)
    latitude_l.append(latitude)
    longitude_l.append(longitude)
    
    # Monitor Progress
    print(address)
    print(latitude, longitude)
    print(c)

<h5>Create New DataFrame, Populate and View</h5>

In [4]:
data = {'Look-up Address': address_l,
        'Latitude': latitude_l,
        'Longitude': longitude_l
        }

temp = pd.DataFrame(data)
temp.head()

,Look-up Address,Latitude,Longitude
0,"19 Shortland Street, Auckland 1010",-36.8468,174.767
1,"109 Queen Street, Auckland 1010",-36.8546,174.763
2,"10/26 Jellicoe Street, Auckland 1010",-36.9544,174.843
3,"18-Customs Street East, Auckland 1010",-36.8447,174.767
4,"8 Wellesley Street West, Auckland 1010",-36.8509,174.764


<h5>Append New DataFrame to Existing DataFrame, View and Save to Excel File</h5>

In [5]:
akl_cbd_biz = pd.concat([akl_cbd_biz, temp], axis=1)

akl_cbd_biz['Precinct'] = akl_cbd_biz['Precinct'].str.strip()    # Remove line breaks

akl_cbd_biz.to_excel(
    r'FULL FILE PATH\FILE.xlsx', index=False, header=True)  # Saved as "address_to_lat_lon.xlsx"

print("Saved to excel file")

akl_cbd_biz.head()

Saved to excel file


,Name,Main Category,Sub Category 1,Sub Category 2,Address Line 1,Address Line 2,Precinct,Post Code,Locality,Phone,Web URL,Web URL COVID Info,Look-up Address,Latitude,Longitude
0,100 Fix,Retail,Electronics and technology,,,19 Shortland Street,Federal & Elliott,1010,Auckland,,,No URL,"19 Shortland Street, Auckland 1010",-36.8468,174.767
1,109 Doctors,Services,Health and wellbeing,Doctors,Level 9,109 Queen Street,Queen Street,,Auckland,(09)3660109,,No URL,"109 Queen Street, Auckland 1010",-36.8546,174.763
2,16 Tun,Hospitality,Bars and clubs,Restaurants,,10/26 Jellicoe Street,Viaduct & Waterfront,,Auckland,09 368 7712,www.16tun.com,URL Parse Error,"10/26 Jellicoe Street, Auckland 1010",-36.9544,174.843
3,1Above Ltd,Services,Health and wellbeing,,Old Sofrana House Level 1,18-Customs Street East,Viaduct & Waterfront,,Auckland,09 379 5708,www.fly1above.com,No Obvious COVID Info,"18-Customs Street East, Auckland 1010",-36.8447,174.767
4,2 Degrees,Retail,Electronics and technology,,Civic Centre,8 Wellesley Street West,Federal & Elliott,,Auckland,09 366 1584,www.2degreesmobile.co.nz,No Obvious COVID Info,"8 Wellesley Street West, Auckland 1010",-36.8509,174.764
